In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
fb = pd.read_excel('sampledataFB\custom-data-Facebook 45d.xlsx', encoding="UTF-8")
imcolFB = fb[['Ref_ID', 'Name', 'Message']]

In [196]:
# uncomment the command below to install ICU engine for word tokenizing
# !pip install PyICU

# tokenize for sentenses and words
from pythainlp.tokenize import sent_tokenize, word_tokenize
from pythainlp.spell import correct

# for pipeline function, an input is a dataframe and it should contains at least 2 columns which is ('Ref_ID', 'Message')
def pipeline(df): 
    result = getname(geturl(getphonenum(getaccountnum(df))))
    try:
        result = result[['Ref_ID', 'Name', 'URLs', 'Phone_No', 'Account_No', 'Account_Name']]
    except:
        result = result[['Ref_ID', 'URLs', 'Phone_No', 'Account_No', 'Account_Name']]
    return result

# get URLs column
def geturl(df):
    result = df.copy()
    urls = []
    for text in result['Message']:
        try:
            tmp = re.findall(r'(https?://\S+)', text)
            if re.findall(r'(https?://\S+)', text) != []:
                urls.append(tmp)
            else:
                urls.append('-')
        except:
            urls.append('-')
    result['URLs'] = urls
    return result

# get phone numbers column
def getphone_no(df):
    result = df.copy()
    phonenumber = []
    firstpt = r'((09|08|06)[0-9][- ][0-9]{3}[- ][0-9]{4})'
    secpt = r'((09|08|06)[- ][0-9]{4}[- ][0-9]{4})'
    trdpt = r'((09|08|06)[0-9][- ][0-9]{7})'
    for text in result['Message']:
        tmp = []
        try:
            first = re.findall(firstpt, text)
            second = re.findall(secpt, text)
            third = re.findall(trdpt, text)
            comb = first+second+third
            tmp = [elem[0] for elem in comb]
            phonenumber.append(tmp if tmp != [] else '-')
        except:
            phonenumber.append('-')
    result['Phone_No'] = phonenumber
    return result
                
# get Line ID column
def getline_id(df):
    result = df.copy()
    regex = r'(?i)((line)|(ไลน์)|(ไลน))[^a-z]'
    line_id = []
    for text in result['Message']:
        try:
            tmp = re.finditer(regex, text)
            tmp = [text[elem.span(0)[0]:elem.span(0)[1]+30] for elem in tmp]
            line_id.append(tmp if tmp != [] else '-')
        except:
            line_id.append('-')
    result['Line_ID'] = line_id
    return result
    
# get account number column and extra column
def getaccount_no(df):
    result = df.copy()
    account_no = []
    extracol = []
    firstpt = r'([0][2689][0-9][- ][0-9]{3}[- ][0-9]{3,4})'
    secpt = r'([0][2689][- ][0-9]{3,4}[- ][0-9]{4})'
    trdpt = r'([0][2689][0-9][- ][0-9]{7})'
    for text in result['Message']:
        try:
            allnum = re.findall(r'([0-9\-]{10,13})', text)
            final = allnum.copy()
            tmp = []
            index = 0
            for num in allnum:
                if (re.search(firstpt, num) != None) or (re.search(secpt, num) != None) or (re.search(trdpt, num) != None) or (re.search(r'[0][23457][-][0-9]{7}', num) != None) or (re.search(r'[0][23457][0-9][-][0-9]{6}', num) != None):
                    final.remove(num)
                elif re.search(r'([0][689][0-9]{8})', num) != None:
                    tmp.append(num)
                    final.remove(num)
                elif re.search(r'([- ]{4,13})', num) != None:
                    final.remove(num)
            account_no.append(final if final != [] else '-')
            extracol.append(tmp if tmp != [] else '-')
        except:
            account_no.append('-')
            extracol.append('-')
    result['Account_No'] = account_no
    result['Extra'] = extracol
    return result

# get account name column
def getname(df):
    result = df.copy()
    Account_name = []
    company_re = r'((บริษัท)|(บจก.)|(บมจ.)|(บจ.))'
    name_re = r'((นาย)|(นางสาว)|(เด็กหญิง)|(นาง)|(เด็กชาย)|(น[.]ส[.])|(ด[.]ญ[.])|(ด[.]ช[.]))'
    for text in result['Message']:
        try:
            sentenses = text.split()
            index = 0
            for i in sentenses:
                if re.search(r'(ชื่อบัญชี)', i) != None:
                    if len(i) <= len('ชื่อบัญชี')+10:
                        if re.search(company_re, sentenses[index+1]) != None:
                            if re.search(r'(จำกัด)', sentenses[index+3]) != None:
                                if re.search(r'(มหาชน)', sentenses[index+4]) != None:
                                    Account_name.append(sentenses[index+1] + ' ' + sentenses[index+2] + ' ' + sentenses[index+3] + sentenses[index+4])
                                    break
                                else:
                                    Account_name.append(sentenses[index+1] + ' ' + sentenses[index+2] + ' ' + sentenses[index+3])
                                    break
                            elif re.search(r'(จำกัด)', sentenses[index+4]) != None:
                                if re.search(r'(มหาชน)', sentenses[index+5]) != None:
                                    Account_name.append(sentenses[index+1] + ' ' + sentenses[index+2] + ' ' + sentenses[index+3] +' '+ sentenses[index+4]+sentenses[index+5])
                                    break
                                else:
                                    Account_name.append(sentenses[index+1] + ' ' + sentenses[index+2] + ' ' + sentenses[index+3] + ' ' + sentenses[index+4])        
                                    break
                            else: 
                                Account_name.append(sentenses[index+1] + ' ' + sentenses[index+2])
                                break
                        elif re.search(name_re, sentenses[index+1]) != None:
                            Account_name.append(sentenses[index+1] + ' ' + sentenses[index+2])
                            break
                        else:
                            Account_name.append(sentenses[index+1] + ' ' + sentenses[index+2] + ' ' + sentenses[index+3])
                            break
                    else:
                        Account_name.append(i)
                        break
                else:
                    if index+1 != len(sentenses):
                        index += 1
                    else:
                        Account_Name.append('-')
        except:
            Account_name.append('-')
    result['Account_Name'] = Account_name
    return result
            
                                
# get bank name column
def getbankname(df):
    result = df.copy()


In [197]:
a = getaccount_no(getphone_no(geturl(getname(imcolFB))))
a[a['Account_Name'] != '-']
#a = getaccount_no(imcolFB)
#a[a['Account_No'] != '-']

,Ref_ID,Name,Message,Account_Name,URLs,Phone_No,Account_No,Extra
823,369236784797_10158429756629798,ไร่ปลูกรัก RAI PLUK RAK Thai Organic Farmer,Organic Cooking แกงเหลืองผัดหอยเชลล์สร้างภูมิค...,บริษัท ไทยออแกนนิค ฟู้ด จำกัด,-,-,[066-267093-2],-
843,369236784797_10158429695364798,ไร่ปลูกรัก RAI PLUK RAK Thai Organic Farmer,Organic Cooking แกงเหลืองผัดหอยเชลล์สร้างภูมิค...,บริษัท ไทยออแกนนิค ฟู้ด จำกัด,-,-,[066-267093-2],-
1796,218503268201212_3335359296515578,ครูวันดีดอทคอม - เปิดสอบครู สมัครงานราชการ,🎯หนังสือสรุปเตรียมสอบครูผู้ช่วยกรณีพิเศษ2563 🔹...,น.ส.พรพรรณ ศรีคำภา,[https://m.me/warasin.dekpua],-,[9830021572],-
1973,369236784797_10158426376124798,ไร่ปลูกรัก RAI PLUK RAK Thai Organic Farmer,Organic Cooking แกงเหลืองผัดเนื้อหอยลาย .... ...,บริษัท ไทยออแกนนิค ฟู้ด จำกัด,-,-,[066-267093-2],-
2284,316245801805608_304803670916822,Vintagebagbyjeab,มาไว ไปไว ของแท้ทุกใบนะคะ ผู้แชร์ส่งฟรี ใครไม่...,อัมไพ ยืนยง เท่านั้น,-,-,-,-
3983,111227565590149_3148119365234272,จังหวัดเชียงใหม่ CM108.com,📣💚 ตั๋วหู้ก่อ ไลน์แมน (LINE MAN) พร้อมให้บริกา...,ใช้ชื่อบัญชีเดียวกับไลน์ส่งข้อความ,[https://lineman.onelink.me/1N3T/25651caa],-,-,-
5940,187258916388_10157029191116389,The Darkest Romance,สวัสดีครับ เราขออนุญาตมาแจ้งให้ทราบว่า ในที่สุ...,นายธิติวัฒน์ รองทอง,-,-,[772-2-31357-1],-
7037,1253942961362212_3152930461463443,ก้าว,ตลาดใจวันสุดท้าย มาร่วมบ๊ายบายความทรงจำ ตลอด 3...,มูลนิธิก้าวคนละก้าว 111-393895-2 ไม่สามารถออกใ...,-,"[086-337-7687, 091-803-3013, 064-806-8065]",[111-393895-2],-
7480,369236784797_10158409543474798,ไร่ปลูกรัก RAI PLUK RAK Thai Organic Farmer,แกงเหลืองผัดหมึกไข่ อุดมด้วยขมิ้นสร้างภูมิคุ้ม...,บริษัท ไทยออแกนนิค ฟู้ด จำกัด,-,-,[066-267093-2],-
8024,127434041553_10159109574396554,AIS,รายการเอไอเอส คนเก่งหัวใจแกร่ง พบน้อง “น้ำหวาน...,ด.ญ.ศจิกา รสชุ่ม,[https://bit.ly/2NTKojT],-,[744-0-44967-3],-


In [191]:
imcolFB['Message'].iloc[31947]

'บอกบุญครับ.. ฝากพี่ ๆ ช่วยเหลือ ช่วยแชร์ ช่วยอุดหนุนสินค้า เพื่อหาเงินดูแลภรรยาที่ป่วยด้วยครับ  ข้อความจาก Facebook Valinthorn Chathaisong  ภรรยาผม น.ส.ชลธิชา บุญพจน์ อายุ 35 ปี เป็นผู้ป่วยนอนติดเตียงเนื่องจาก เส้นเลือดในสมองแตกภายหลังมาเสมหะอุดตันท่อที่เจาะคอเพื่อให้หายใจ ทำให้หัวใจหยุดเต้นไปเกือบ 10 นาที ต้องปั้มหัวใจขึ้นมา ทำให้ไม่มีออกซิเจนไปเลี้ยงสมองทำให้สมองตายไม่สามารถตอบสนองได้ ทุกวันนี้หมอก็ดูแลตามสภาพ ส่วนผมก็ดูแลภรรยา อาบน้ำ ทำแผลกดทับ ป้อนอาหารและคอยดูดเสมหะ  ตอนนี้ต้องมาอาศัยอยู่กับแม่ของผมที่บ้าน ผมไม่สามารถไปทำงานที่ไหนไกล ๆ นาน ๆ ได้นอกจากวันหยุดจะให้ลูกสาวผมดูแลแทน ถ้าไม่มีงานก็ช่วยแม่ทำขนมขายที่บ้าน มีรายได้แล้วแต่แม่จะแบ่งให้ ประกอบกับช่วงนี้ที่โควิดระบาด ยิ่งทำให้ลำบาก เดือดร้อนเข้าไปใหญ่ จึงอยากขอความเมตตาจากท่านผู้ใจบุญ ช่วยเหลือช่วยสนับสนุน สินค้า ที่พอจะมาเป็นค่าใช้จ่ายในการดูแลครอบครัวได้บ้าง  ผมขายจั๊บเลี้ยงเมียป่วยติดเตียงกับลูกครับ 🍜ของแซ่บเมืองอุบล ต้องจั๊บอุบลสูตรคุณย่า🍜 ต้มก็ได้ เวฟก็ได้ ง่ายๆเพียง 5 นาที ♨️♨️♨️♨️⭐️⭐️⭐️⭐️  💋โปรพิเศษ 10 ห่อ เพียง 299 บาท

In [177]:
z=re.findall(r'[0][2689][0-9][- ][0-9]{3}[- ][0-9]{4}', imcolFB['Message'].iloc[13196])
z

['086-337-7687', '091-803-3013', '064-806-8065']

In [129]:
getline_id(imcolFB)

,Ref_ID,Name,Message,Line_ID
0,393668090664206_3203512436346410,Kociety,🚕 ปลดล็อคเฟส 3 มาเป็นวันที่ 4 แล้วนะคะ เพื่อนๆ...,[Line Official ID: @kociety 📱: 0-396]
1,746280992139796_2636489409785602,unknown,วันนี้มื้อเย็นสั่งไลน์แมนซื้อไก่ทอดหาดใหญ่มากิ...,"[ไลน์แมนซื้อไก่ทอดหาดใหญ่มากินที่บ้า, ไลน์แมนม..."
2,607423482637768_3188984437814980,ทรายแมว,มาเเล้วจร้า ทรายเพ็ทเฮง ไม้สนซีด้า สีขาว มีกลิ...,-
3,796544413703953_4130486343643060,ของเล่นและโมเดลInW HOBBY & TOYS,เปิดจอง ผู้ผลิต Kotobukiya วางจำหน่ายเดือน10-2...,[ไลน์ yamapa07 - อีเมล inwhobbytoys@]
4,165617276938555_1556194991214103,ST Watch And Time - ทรัพย์ทวีนาฬิกา กาญจนบุรี,🚩นาฬิกา CASIO สายหนัง ขนาด38มม.และ28มม. 🙏🏼พร้อ...,-
5,164899896970695_2811449822315676,Mushroom Club,🌷N E W 🌷 เสื้อผ้าคอตต้อน ลายจุด ทรงแขนตุ๊กตา ง...,[Line ID: @mushroomclub (มี @ ด้วยนะ]
6,118605731643950_1503271256510717,ห้างทองทองทวีทรัพย์ - Thongtaveesap Goldsmith,สร้อยคอลายเกลียวคริสตัลห้อยหัวใจคิวปิด ตัวจี้ท...,-
7,118605731643950_1503271823177327,ห้างทองทองทวีทรัพย์ - Thongtaveesap Goldsmith,สร้อยคอลายปะคำสายฝนสองกษัตริย์ห้อยหัวใจไฮโซ เร...,-
8,362876520513488_1930762443724880,Mamy Ann แฟชั่นเสื้อผ้าคนท้อง,❤ ชุดคลุมท้องใส่ออกงานผ้าพลีท สไตล์เรียบหรู ดู...,[LINE OPEN CHAT https://bit.ly/32GQY]
9,1745101772385932_2727371167492316,Sarakadee Magazine,14 เล่มผลงานล้ำค่า ของปรมาจารณ์ ด้านโบราณคดีแล...,[Line : @sarakadeemag inbox : http:/]


In [83]:
x = [1,2]
if x[0] == 1:
    print(x[1])
elif x[1] == 2:
    print(x[0])

2


In [143]:
re.search(r'[0-9\-]{10,13}', imcolFB['Message'].iloc[7037]).span(0)[1]

577

In [148]:
x= [1,2,1,2]
y = 2 
if y in x:
    x.remove(y)
print(x)

[1, 1, 2]


In [156]:
x = [1,2,3]
x.remove(1)
x

[2, 3]